# **Prueba Analistas**

In [1]:
# importar los datos

import pandas as pd
import numpy as np

# Probar con una codificación diferente
datos = pd.read_csv('analista_de_operaciones.csv', delimiter=';', encoding='latin1', header=0)

print(datos.shape)
print(datos.head())


(105388, 1)
  REGIONAL,CIUDAD,CODTRABAJO,FECHA,CUENTA,ORDEN,CANTIDAD,NOMBRE TECNICO REAL,CEDULA TECNICO REAL,TIPO_TRABAJO,PRECIO DE TRABAJO,TIPO DE MOVIL,ESQUEMA DE TRABAJO,COORDINADOR
0  CENTRO,BOGOTA,VENTEC,01/07/2023,15589183,38053...                                                                                                                        
1  CENTRO,BOGOTA,NROUTER,01/07/2023,15434166,3805...                                                                                                                        
2  CENTRO,BOGOTA,NROUTER,01/07/2023,15581974,3805...                                                                                                                        
3  CENTRO,BOGOTA,NROUTER,01/07/2023,15634211,3805...                                                                                                                        
4  CENTRO,BOGOTA,NROUTER,01/07/2023,15589183,3805...                                                                       

In [2]:
# tipo de datos

type(datos)

pandas.core.frame.DataFrame

In [3]:
# cantidad de datos 

datos.shape

(105388, 1)

In [4]:
# las columnas

datos.columns

Index(['REGIONAL,CIUDAD,CODTRABAJO,FECHA,CUENTA,ORDEN,CANTIDAD,NOMBRE TECNICO REAL,CEDULA TECNICO REAL,TIPO_TRABAJO,PRECIO DE TRABAJO,TIPO DE MOVIL,ESQUEMA DE TRABAJO,COORDINADOR'], dtype='object')

## **Promedio Servicio**

In [5]:
# promedio de trabajo

promedio_precio = datos['PRECIO DE TRABAJO'].mean()
promedio_precio

KeyError: 'PRECIO DE TRABAJO'